# Consolidated Betting Strategy
This notebook combines previous analysis techniques for double chance and over/under accumulator bets.

In [ ]:
import itertools
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

## Example Match Data
Replace or extend this list with your own probabilities, odds, and expected goals.

In [ ]:
matches = [
    {'match': 'Club Brugge vs Aston Villa', 'home_team':'Club Brugge', 'away_team':'Aston Villa', 'home_win_prob':0.286, 'draw_prob':0.294, 'away_win_prob':0.5, 'home_win_odds':3.5, 'draw_odds':3.4, 'away_win_odds':2.0, 'home_xg':1.2, 'away_xg':1.8},
    {'match': 'Shakhtar Donetsk vs Young Boys', 'home_team':'Shakhtar Donetsk', 'away_team':'Young Boys', 'home_win_prob':0.476, 'draw_prob':0.303, 'away_win_prob':0.278, 'home_win_odds':2.1, 'draw_odds':3.3, 'away_win_odds':3.6, 'home_xg':1.5, 'away_xg':1.1},
    {'match': 'Bayern Munich vs Benfica', 'home_team':'Bayern Munich', 'away_team':'Benfica', 'home_win_prob':0.714, 'draw_prob':0.211, 'away_win_prob':0.133, 'home_win_odds':1.4, 'draw_odds':4.75, 'away_win_odds':7.5, 'home_xg':2.3, 'away_xg':0.8},
    {'match': 'Crvena Zvezda vs Barcelona', 'home_team':'Crvena Zvezda', 'away_team':'Barcelona', 'home_win_prob':0.167, 'draw_prob':0.25, 'away_win_prob':0.667, 'home_win_odds':6.0, 'draw_odds':4.0, 'away_win_odds':1.5, 'home_xg':0.9, 'away_xg':2.1},
    {'match': 'Feyenoord vs FC Salzburg', 'home_team':'Feyenoord', 'away_team':'FC Salzburg', 'home_win_prob':0.444, 'draw_prob':0.286, 'away_win_prob':0.323, 'home_win_odds':2.25, 'draw_odds':3.5, 'away_win_odds':3.1, 'home_xg':1.6, 'away_xg':1.3},
    {'match': 'Inter Milan vs Arsenal', 'home_team':'Inter Milan', 'away_team':'Arsenal', 'home_win_prob':0.4, 'draw_prob':0.303, 'away_win_prob':0.357, 'home_win_odds':2.5, 'draw_odds':3.3, 'away_win_odds':2.8, 'home_xg':1.4, 'away_xg':1.2},
    {'match': 'PSG vs Atletico Madrid', 'home_team':'PSG', 'away_team':'Atletico Madrid', 'home_win_prob':0.556, 'draw_prob':0.278, 'away_win_prob':0.222, 'home_win_odds':1.8, 'draw_odds':3.6, 'away_win_odds':4.5, 'home_xg':1.7, 'away_xg':0.9},
    {'match': 'Sparta Prague vs Brest', 'home_team':'Sparta Prague', 'away_team':'Brest', 'home_win_prob':0.625, 'draw_prob':0.25, 'away_win_prob':0.182, 'home_win_odds':1.6, 'draw_odds':4.0, 'away_win_odds':5.5, 'home_xg':1.8, 'away_xg':0.7}
]


## Helper Functions

In [ ]:
from bet_builder_utils import calculate_double_chance_labels, generate_accumulator_df, highlight_best_bets


In [ ]:
for match in matches:
    (home_draw_label, home_draw), (away_draw_label, away_draw), (home_away_label, home_away) = calculate_double_chance_labels(
        match['home_team'], match['away_team'], match['home_win_prob'], match['draw_prob'], match['away_win_prob']
    )
    match['home_draw'] = (home_draw_label, home_draw)
    match['away_draw'] = (away_draw_label, away_draw)
    match['home_away'] = (home_away_label, home_away)
    match['estimated_goals'] = round(match['home_xg'] + match['away_xg'], 2)


### Generate Accumulator Combinations

In [ ]:
# generate_accumulator_df imported from bet_builder_utils


In [ ]:
acca_df = generate_accumulator_df(matches)
acca_df.sort_values('Expected Value', ascending=False, inplace=True)
acca_df.head()


### Highlight Best Bets

In [ ]:
# highlight_best_bets imported from bet_builder_utils


### Expected Value vs Probability Visualization

In [ ]:
x = acca_df['Cumulative Probability'].values
y = acca_df['Expected Value'].values
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(x.reshape(-1,1), y)
cv = cross_val_score(model, x.reshape(-1,1), y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated MSE: {-cv.mean():.4f}')

x_new = np.linspace(x.min(), x.max(), 200)
y_gb = model.predict(x_new.reshape(-1,1))

plt.figure(figsize=(8,6))
plt.scatter(x, y, c=acca_df['Cumulative Odds'], cmap='viridis', s=20, alpha=0.6)
plt.plot(x_new, y_gb, color='magenta', label='Gradient Boosting')
plt.colorbar(label='Cumulative Odds')
plt.xlabel('Cumulative Probability')
plt.ylabel('Expected Value')
plt.title('Expected Value vs Probability')
plt.legend()
plt.show()


### Selecting Example 6-Team Accumulator

In [ ]:
six_team = matches[:6]
acca_6 = generate_accumulator_df(six_team)
acca_6.sort_values('Expected Value', ascending=False, inplace=True)
acca_6.head(20)


### Selecting Example 8-Team Accumulator

In [ ]:
acca_8 = generate_accumulator_df(matches)
acca_8.sort_values('Expected Value', ascending=False, inplace=True)
acca_8.head(20)


This notebook can be extended with your own match data and tuned filtering parameters to suit different risk strategies.